<a href="https://colab.research.google.com/github/JeroAgullo/fundamental_valuation/blob/main/fundamental_valuation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fundamental Valuation using Python

## Introduction

**Project Approach**  
This project provides a fundamental valuation of NVIDIA Corporation (NVDA) using Python tools and financial data sourced from the Yahoo Finance API via the **yfinance** library.

The notebook parses and standardizes key financial statement elements into a format that facilitates ratio analysis and valuation modeling.

**Fundamental Analysis**  
Fundamental analysis is a method used in accounting and finance to estimate the **intrinsic value of a security** in order to help with investment decisions.

Unlike technical analysis, which focuses on price trends, fundamental analysis seeks to determine the "true" value of a security to identify whether it is overvalued, undervalued, or fairly valued compared with its current market price.

## First steps

We will start by importing the needed libraries. Then, we are creating an object for the chosen company using yfinance's function '.Ticker', allowing us to get all its public financial information.

We are using NVIDIA's ticker (the market "ID") for this project, but any other ticker can be applied. Here is a list of companies in the S&P500 Index, and their ticker: https://en.wikipedia.org/wiki/List_of_S%26P_500_companie

In [10]:
# Libraries import
import yfinance as yf
import pandas as pd

In [11]:
# NVIDIA object creation
nvda = yf.Ticker("NVDA")

In [12]:
# Connection check
headers = nvda.financials.index.tolist() # We will print the headers of the Income Statement (Financials) report
for h in headers:
  print(h)

Tax Effect Of Unusual Items
Tax Rate For Calcs
Normalized EBITDA
Total Unusual Items
Total Unusual Items Excluding Goodwill
Net Income From Continuing Operation Net Minority Interest
Reconciled Depreciation
Reconciled Cost Of Revenue
EBITDA
EBIT
Net Interest Income
Interest Expense
Interest Income
Normalized Income
Net Income From Continuing And Discontinued Operation
Total Expenses
Total Operating Income As Reported
Diluted Average Shares
Basic Average Shares
Diluted EPS
Basic EPS
Diluted NI Availto Com Stockholders
Net Income Common Stockholders
Net Income
Net Income Including Noncontrolling Interests
Net Income Continuous Operations
Tax Provision
Pretax Income
Other Income Expense
Other Non Operating Income Expenses
Special Income Charges
Restructuring And Mergern Acquisition
Net Non Operating Interest Income Expense
Interest Expense Non Operating
Interest Income Non Operating
Operating Income
Operating Expense
Research And Development
Selling General And Administration
Gross Profit

## Data extract

We are now getting all the necessary data to proceed with the fundamental analysis. Not all the accessible information is needed, so we are selecting specific one from each Financial State:
- Income Statement (Financials)
- Balance Sheet
- Cashflow

In [13]:
# Auxiliar function to get only existing values and to limit error cases
def safe_extract(df, row_name, column_name):
    if row_name in df.index:
        return df.loc[row_name, column_name]
    else:
        return None

In [14]:
# Data extract - Income Statement
df = nvda.financials
column_name = df.columns[1]

total_revenue = safe_extract(df, "Total Revenue", column_name)
gross_profit = safe_extract(df, "Gross Profit", column_name)
ebitda = safe_extract(df, "EBITDA", column_name)
ebit = safe_extract(df, "EBIT", column_name)
net_income = safe_extract(df, "Net Income", column_name)

In [15]:
# Data extract - Balance Sheet
df= nvda.balance_sheet
column_name = df.columns[0]

total_assets = safe_extract(df, "Total Assets", column_name)
total_lianilities = safe_extract(df, "Total Liabilities Net Minority Interest", column_name)
total_equity = safe_extract(df, "Total Equity Gross Minority Interest", column_name)
cash = safe_extract(df, "Cash And Cash Equivalents", column_name)

In [16]:
# Data print
print("Income Statement results:")
print(f"Total Revenue: {total_revenue:,.0f}$")
print(f"Total Profit: {gross_profit:,.0f}")
print(f"EBITDA: {ebitda:,.0f}$")
print(f"EBIT: {ebit:,.0f}$")
print(f"Net Income: {net_income:,.0f}$")

print("\nBalance Sheet results:")
print(f"Total Assets: {total_assets:,.0f}$")
print(f"Total Liabilities: {total_lianilities:,.0f}$")
print(f"Total Equity: {total_equity:,.0f}$")
print(f"Cash: {cash:,.0f}$")

Income Statement results:
Total Revenue: 60,922,000,000$
Total Profit: 44,301,000,000
EBITDA: 35,583,000,000$
EBIT: 34,075,000,000$
Net Income: 29,760,000,000$

Balance Sheet results:
Total Assets: 111,601,000,000$
Total Liabilities: 32,274,000,000$
Total Equity: 79,327,000,000$
Cash: 8,589,000,000$


## Financial Ratios

In [ ]:
roe = net_income / total_equity
print(f"ROE: {roe:.2%}")

cash_ratio = cash / total_assets
print(f"Cash Ratio: {cash_ratio:.2%}")

debt_ratio = total_lianilities / total_assets
print(f"Debt Ratio: {debt_ratio:.2%}")

profit_revenue = gross_profit / total_revenue
print(f"Profit (as % of Revenue): {profit_revenue:.2%}")

ebitda_revenue = ebitda / total_revenue
print(f"EBITDA (as % of Revenue): {ebitda_revenue:.2%}")

ROE: 37.52%
Cash Ratio: 7.70%
Debt Ratio: 28.92%
Profit (as % of Revenue): 72.72%
EBITDA (as % of Revenue): 58.41%
